## Scraped json files to images
As the file name suggests, this is a notebook to get the scraped links (in .json files) and convert them to images.<br>The steps are simple:<br>
1) Import the modules<br>
2) Prepare the file paths<br>
3) Open the **.json** file<br>
4) Create a **folder** for that class<br>
5) Get the **image**<br>
6) Crop & save the **face(s)** on the image<br>
7) If an **error** occurs, print it to the console<br>

In [1]:
import cvlib as cv
import numpy as np
import io, os, json
import cv2, requests
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
%%time
json_list = ("sad_img_url.json", 
             "shocked_img_url.json", 
             "happy_img_url.json", 
             "neutral_img_url.json")

folder_list = ("sad_folder", 
               "shocked_folder", 
               "happy_folder", 
               "neutral_folder")

for json_file, folder in zip(json_list, folder_list):
    with open(json_file, "r") as f:
        loaded_json = json.load(f)
        print(f"{folder} is filling...")
        os.mkdir(folder)    # create the folder and fill it!
        for i in range(len(loaded_json)):
            try:
                img_link = loaded_json[i].popitem()[1]
                # Read the link > Get the content (in bytes) > Open the img:
                img = Image.open(io.BytesIO(requests.get(img_link).content))
                img = np.array(img)
                
                # OpenCV requires 3D images to detect faces:
                # So, if not BGR, convert to it (code=8 refers to "COLOR_GRAY2BGR")
                img = cv2.cvtColor(img, code=8) if len(img.shape) != 3 else img
                
                # Detect the faces > Crop the ones with >80% confidence > Save it as a jpg
                faces, confidence = cv.detect_face(img, threshold=0.80)
                for face_num, face in enumerate(faces, start=1):
                    # frame it:
                    (startX, startY) = (face[0], face[1])    # top left corner
                    (endX, endY)     = (face[2], face[3])    # bottom right corner
                    cropped_frame = np.copy(img[startY:endY, startX:endX])
                    # keep only large-enough (24-pixel) frames:
                    if cropped_frame.shape[0] > 23 or cropped_frame.shape[1] > 23:
                        to_path = f"{folder}/p{i}_f{face_num}.jpg"
                        Image.fromarray(cropped_frame).save(to_path)
                        # sample output file: "p15_f2.jpg" (picture15, face2)
            except Exception as e:
                # Don't you wanna know what the errors are?
                print(e, f"(error at index: {i})")
                continue
        print(f"{folder} is filled!\n")
# For this instance, only one of the folders was filled to save time.
# Also you can see some of the errors that popped up:

shocked_folder is filling...
tile cannot extend outside image (error at index: 573)
tile cannot extend outside image (error at index: 1840)
Invalid URL '': No schema supplied. Perhaps you meant http://? (error at index: 1845)
tile cannot extend outside image (error at index: 2045)
tile cannot extend outside image (error at index: 2086)
cannot identify image file <_io.BytesIO object at 0x0000013E87042B80> (error at index: 2203)
tile cannot extend outside image (error at index: 2317)
cannot identify image file <_io.BytesIO object at 0x0000013E86EFC680> (error at index: 2364)
tile cannot extend outside image (error at index: 2369)
cannot identify image file <_io.BytesIO object at 0x0000013E86EFC680> (error at index: 2558)
tile cannot extend outside image (error at index: 2747)
tile cannot extend outside image (error at index: 2778)
shocked_folder is filled!

Wall time: 37min 4s
